In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
import torch.optim as optim
import numpy as np
import random
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.data import Data
from torch_geometric.nn import ARMAConv
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, log_loss
)
from torch.utils.data import TensorDataset, DataLoader, Subset
from torch_geometric.data import Data
from torch_geometric.nn import ARMAConv
from torch.utils.data import TensorDataset, DataLoader, Subset
from torchvision import models

In [19]:
import torch
print("CUDA available:", torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU Name: NVIDIA RTX A4000


In [20]:
# data = np.load('/home/snu/Downloads/breastmnist_224.npz', allow_pickle=True)
# all_images = np.concatenate([data['train_images'], data['val_images'], data['test_images']], axis=0)
# all_labels = np.concatenate([data['train_labels'], data['val_labels'], data['test_labels']], axis=0).squeeze()

# # Convert to 3-channel RGB
# images = all_images.astype(np.float32) / 255.0
# images = np.repeat(images[:, None, :, :], 3, axis=1)  # (N, 3, 224, 224)
# X_torch = torch.tensor(images)
# y_torch = torch.tensor(all_labels).long()
# print(f"Raw images: {X_torch.shape}, Labels: {y_torch.shape}")

In [21]:
data = np.load('/home/snu/Downloads/pneumoniamnist_224.npz', allow_pickle=True)
all_images = np.concatenate([data['train_images'], data['val_images'], data['test_images']], axis=0)
all_labels = np.concatenate([data['train_labels'], data['val_labels'], data['test_labels']], axis=0).squeeze()

# Convert to 3-channel RGB
images = all_images.astype(np.float32) / 255.0
images = np.repeat(images[:, None, :, :], 3, axis=1)  # (N, 3, 224, 224)
X_torch = torch.tensor(images)
y_torch = torch.tensor(all_labels).long()
print(f"Raw images: {X_torch.shape}, Labels: {y_torch.shape}")

Raw images: torch.Size([5856, 3, 224, 224]), Labels: torch.Size([5856])


In [22]:
class0_idx = [i for i in range(len(y_torch)) if y_torch[i] == 0]
class1_idx = [i for i in range(len(y_torch)) if y_torch[i] == 1]

random.seed(42)
sampled_class0 = random.sample(class0_idx, min(2000, len(class0_idx)))
sampled_class1 = random.sample(class1_idx, min(2000, len(class1_idx)))

selected_indices = sampled_class0 + sampled_class1
random.shuffle(selected_indices)

subset_dataset = Subset(TensorDataset(X_torch, y_torch), selected_indices)
subset_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False)

In [23]:
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()
resnet = resnet.to(device)
resnet.eval()

resnet_feats = []
y_list = []
with torch.no_grad():
    for imgs, labels in subset_loader:
        imgs = imgs.to(device)
        feats = resnet(imgs)
        resnet_feats.append(feats.cpu())
        y_list.extend(labels.cpu().tolist())

features = torch.cat(resnet_feats, dim=0).numpy().astype(np.float32)  # shape (N, feat_dim)
y_labels = np.array(y_list).astype(np.float32)
print("Feature shape:", features.shape, "Label shape:", y_labels.shape)

/home/snu/anaconda3/envs/torch_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/snu/anaconda3/envs/torch_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Feature shape: (3583, 512) Label shape: (3583,)


In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
from torch_geometric.nn import ARMAConv

class ARMA_SemiSupervised(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device, num_stacks=1, num_layers=1):
        super().__init__()
        self.device = device
        self.num_clusters = output_dim # Initialize num_clusters

        # Only one ARMAConv layer
        self.conv1 = ARMAConv(input_dim, hidden_dim,
                              num_stacks=num_stacks, num_layers=num_layers,
                              shared_weights=True, dropout=0.2)

        self.dropout = nn.Dropout(0.25)
        self.fc = nn.Linear(hidden_dim, output_dim)

        # Activation function
        activations = {
            "SELU": nnFn.selu,
            "SiLU": nnFn.silu,
            "GELU": nnFn.gelu,
            "RELU": nnFn.relu,
            "ELU": nnFn.elu,
            "PReLU": nnFn.prelu,
            "LeakyReLU": nnFn.leaky_relu
        }
        self.act = activations.get("RELU", nnFn.relu)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Single layer
        x = self.conv1(x, edge_index)
        x = self.act(x)
        x = self.dropout(x)

        logits = self.fc(x)
        return logits

    def cut_loss(self, A, S):
        S = nnFn.softmax(S, dim=1)
        A_pool = torch.matmul(torch.matmul(A, S).t(), S)
        num = torch.trace(A_pool)

        D = torch.diag(torch.sum(A, dim=-1))
        D_pooled = torch.matmul(torch.matmul(D, S).t(), S)
        den = torch.trace(D_pooled)
        mincut_loss = -(num / den)

        St_S = torch.matmul(S.t(), S)
        I_S = torch.eye(self.num_clusters, device=self.device)
        ortho_loss = torch.norm(St_S / torch.norm(St_S) - I_S / torch.norm(I_S))

        return mincut_loss + ortho_loss

In [25]:
def create_adj(F, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    W = np.where(W >= alpha, 1, 0).astype(np.float32)
    W = W / W.max()
    return W

def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats).float()
    edge_index = torch.from_numpy(np.array(np.nonzero((adj > 0))))
    return Data(x=node_feats, edge_index=edge_index)

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
alpha = 0.9
feats_dim = features.shape[1]
hidden_dim = 256
num_classes = 2
num_epochs = 2000
lr = 0.0001
weight_decay = 1e-4
batch_print_freq = 500
lambda_mod = 0.1  #0.2 #0.001  # weight for modularity loss
# lambda_sup = 1.0

In [27]:
W = create_adj(features, alpha)
data = load_data(W, features).to(device)
print(data)

Data(x=[3583, 512], edge_index=[2, 1642039])


In [28]:
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.9, random_state=42)

accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []

for fold, (train_idx, test_idx) in enumerate(sss.split(features, y_labels.astype(np.int64)), start=1):
    print(f"\n=== Fold {fold} ===")

    # Convert to tensors
    train_idx_t = torch.from_numpy(train_idx).long().to(device)
    y_train_tensor = torch.from_numpy(y_labels[train_idx]).long().to(device)
    A_tensor = torch.from_numpy(W).float().to(device)

    # Initialize model
    model = ARMA_SemiSupervised(feats_dim, hidden_dim, num_classes, device).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    ce_loss = nn.CrossEntropyLoss()

    # Training
    for epoch in range(1, num_epochs + 1):
        model.train()
        optimizer.zero_grad()

        logits = model(data)
        loss_sup = ce_loss(logits[train_idx_t], y_train_tensor)
        loss_unsup = model.cut_loss(A_tensor, logits) # Corrected method call
        total_loss = loss_sup + lambda_mod * loss_unsup

        total_loss.backward()
        optimizer.step()

        if epoch % batch_print_freq == 0 or epoch == 1:
            model.eval()
            with torch.no_grad():
                preds_train = logits[train_idx_t].argmax(dim=1)
                acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
            print(f"Fold {fold} Epoch {epoch}: "
                  f"TotalLoss={total_loss.item():.6f} | Sup={loss_sup.item():.6f} | "
                  f"Unsup={loss_unsup.item():.6f} | TrainAcc={acc_train:.4f}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        out = model(data)
        preds = out.argmax(dim=1).cpu().numpy()
        probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()  # Probability for class 1

    y_test = y_labels[test_idx]
    y_pred_test = preds[test_idx]
    y_prob_test = probs[test_idx]

    acc = accuracy_score(y_test, y_pred_test)
    prec = precision_score(y_test, y_pred_test, zero_division=0)
    rec = recall_score(y_test, y_pred_test, zero_division=0)
    f1 = f1_score(y_test, y_pred_test, zero_division=0)
    auc = roc_auc_score(y_test, y_prob_test)
    ce = log_loss(y_test, y_prob_test)

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)
    aucs.append(auc)
    ce_losses.append(ce)

    print(f"Fold {fold} → "
          f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
          f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")


# Final summary
print("\n=== Average Results Across 20 Folds ===")
print(f"Accuracy:  {np.mean(accuracies):.4f} \u00b1 {np.std(accuracies):.4f}")
print(f"Precision: {np.mean(precisions):.4f} \u00b1 {np.std(precisions):.4f}")
print(f"Recall:    {np.mean(recalls):.4f} \u00b1 {np.std(recalls):.4f}")
print(f"F1-score:  {np.mean(f1_scores):.4f} \u00b1 {np.std(f1_scores):.4f}")
print(f"AUC:       {np.mean(aucs):.4f} \u00b1 {np.std(aucs):.4f}")
print(f"CE Loss:   {np.mean(ce_losses):.4f} \u00b1 {np.std(ce_losses):.4f}")


=== Fold 1 ===
Fold 1 Epoch 1: TotalLoss=0.864995 | Sup=0.887268 | Unsup=-0.222736 | TrainAcc=0.4581
Fold 1 Epoch 500: TotalLoss=-0.028553 | Sup=0.052573 | Unsup=-0.811261 | TrainAcc=0.9860
Fold 1 Epoch 1000: TotalLoss=-0.059049 | Sup=0.022075 | Unsup=-0.811247 | TrainAcc=0.9972
Fold 1 Epoch 1500: TotalLoss=-0.072147 | Sup=0.010076 | Unsup=-0.822227 | TrainAcc=1.0000
Fold 1 Epoch 2000: TotalLoss=-0.074910 | Sup=0.009836 | Unsup=-0.847459 | TrainAcc=1.0000
Fold 1 → Acc=0.9346 | Prec=0.9749 | Rec=0.9061 | F1=0.9392 | AUC=0.9867 | CE Loss=0.2571

=== Fold 2 ===
Fold 2 Epoch 1: TotalLoss=0.851426 | Sup=0.874609 | Unsup=-0.231830 | TrainAcc=0.5000
Fold 2 Epoch 500: TotalLoss=-0.012597 | Sup=0.062997 | Unsup=-0.755941 | TrainAcc=0.9804
Fold 2 Epoch 1000: TotalLoss=-0.041602 | Sup=0.035181 | Unsup=-0.767830 | TrainAcc=0.9832
Fold 2 Epoch 1500: TotalLoss=-0.062560 | Sup=0.014847 | Unsup=-0.774068 | TrainAcc=0.9944
Fold 2 Epoch 2000: TotalLoss=-0.068539 | Sup=0.008824 | Unsup=-0.773631 | Train